## MBTiles
Be vary of performing any `geometry` calculations on the dataset once it's out o the EPSG:27700 format, as it'll be way less accurate in the `geometry.area()` or even represent it in degrees instead o meters, as per the CRS UTM zone.

It's likely possible to use feature aggregation against area_ha to entirely bypass having to use Area Calculation, but it's not clear if it'll result in a performant and size-efficient MBTiles file.

### Notes

All runtimes are indicated for Apple M1 Max 64GB. 

In [ ]:
import os
import geopandas as gpd
gpd.options.io_engine = 'pyogrio'

In [ ]:
awi_attribution_html = 'Natural England, NatureScot, Scottish Forestry, Natural Resources Wales. Contains Ordnance Survey Data. © <a href="https://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/">Crown copyright and database right 2006-2022</a>'
nfi_attribution_html = 'Forestry Commission. Contains Ordnance Survey Data. © <a href="https://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/">Crown copyright and database right 2012-2022</a>'
combined_attribution_html = 'Forestry Commission, Natural England, NatureScot, Scottish Forestry, Natural Resources Wales. Contains Ordnance Survey Data. © <a href="https://www.nationalarchives.gov.uk/doc/open-government-licence/version/3/">Crown copyright and database right 2006-2022</a>'

In [ ]:
def generate_tiles(year, datasetname, description, attribution):
    p1 = f'tiles/{datasetname}_{year}-p1.mbtiles'
    p2 = f'tiles/{datasetname}_{year}-p2.mbtiles'
    p3 = f'tiles/{datasetname}_{year}-p3.mbtiles'
    p4 = f'tiles/{datasetname}_{year}-p4.mbtiles'
    p12 = f'tiles/{datasetname}_{year}-p12.mbtiles'
    p34 = f'tiles/{datasetname}_{year}-p34.mbtiles'
    fin = f'tiles/{datasetname}_{year}.mbtiles'
    file_names = [p1, p2, p3, p4, p12, p34]

    !tippecanoe -P -z5 --gamma=2 -o '{p1}' 'tiles/{datasetname}_{year}.geojson' -n '{datasetname}_{year}' -l '{datasetname}' --attribution='{attribution}' --description='{description}' -f
    !tippecanoe -P -Z6 -z6 --gamma=1 --drop-smallest-as-needed -o '{p2}' 'tiles/{datasetname}_{year}.geojson' -n '{datasetname}_{year}' -l '{datasetname}' --attribution='{attribution}' --description='{description}' -f
    !tippecanoe -P -Z7 -zg -o '{p3}' 'tiles/{datasetname}_{year}.geojson' -n '{datasetname}_{year}' -l '{datasetname}' --attribution='{attribution}' --description='{description}' -f
    
    !tile-join -pk --force -o $p12 $p1 $p2
    !tile-join -pk --force -o $fin $p12 $p3

    for file in file_names:
        if os.path.exists(file):
            os.remove(file)

In [ ]:
# Runtime: 2m, RAM: 3GB max, Disk: 55MB
awi_dataset = gpd.read_parquet('../data/processed/gb_awi_dataset.parquet')
awi_dataset.geometry = awi_dataset.geometry.to_crs(epsg=4326)
awi_dataset.geometry = awi_dataset.geometry.simplify(1e-5)
awi_dataset.to_file('tiles/gb_awi_dataset.geojson', driver='GeoJSONSeq')
!tippecanoe -zg -o 'tiles/gb_awi_dataset.mbtiles' 'tiles/gb_awi_dataset.geojson' -n 'gb_awi_dataset' -l 'gb_awi_dataset' --attribution='{awi_attribution_html}' --description='Ancient Woodland Inventories datasets combined across Great Britain' -f
os.remove('tiles/gb_awi_dataset.geojson')
del awi_dataset

In [ ]:
# Runtime per year: 10m, RAM: 12GB max, Disk: 220-400MB per year, total 3GB
for year in range(2012, 2023):
    nfi_awi_overlay = gpd.read_parquet(f'processed/gb_nfi_awi_overlay_{year}.parquet')
    nfi_awi_overlay.geometry = nfi_awi_overlay.geometry.to_crs(epsg=4326)
    nfi_awi_overlay.geometry = nfi_awi_overlay.geometry.simplify(1e-5)
    nfi_awi_overlay.to_file(f'tiles/gb_nfi_awi_overlay_{year}.geojson', driver='GeoJSONSeq')
    generate_tiles(year, 'gb_nfi_awi_overlay', f'National Forest Inventory dataset for {year} overlayed with the Ancient Woodland Inventories datasets combined across Great Britain', combined_attribution_html)
    os.remove(f'tiles/gb_nfi_awi_overlay_{year}.geojson')

del nfi_awi_overlay

In [ ]:
# Runtime, per year: 6m, RAM: 8GB max, Disk: 135-150MB, total 1.6GB
for year in range(2012, 2023):
    nfi_dataset = gpd.read_parquet(f'processed/gb_nfi_dataset_{year}.parquet')
    nfi_dataset.geometry = nfi_dataset.geometry.to_crs(epsg=4326)
    nfi_dataset.geometry = nfi_dataset.geometry.simplify(1e-5)
    nfi_dataset.to_file(f'tiles/gb_nfi_dataset_{year}.geojson', driver='GeoJSONSeq')
    generate_tiles(year, 'gb_nfi_dataset', f'National Forest Inventory dataset for {year}', nfi_attribution_html)
    os.remove(f'tiles/gb_nfi_dataset_{year}.geojson')

del nfi_dataset